In [1]:
import setuptools
%load_ext Cython

In [6]:
%env DISTUTILS_USE_SDK=1
%env MSSdk=1

import os
os.environ["PATH"] += r"C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\bin"

env: DISTUTILS_USE_SDK=1
env: MSSdk=1


In [8]:
%%cython -a
import cython
cimport cython

cdef int alpha = 255
print(alpha)

CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\VC\\bin\\cl.exe' failed with exit status 2

In [7]:
import os
os.environ["PATH"].split(";")

['C:\\Program Files\\ImageMagick-7.0.3-Q16',
 'C:\\ProgramData\\Oracle\\Java\\javapath',
 'C:\\Program Files (x86)\\Intel\\iCLS Client\\',
 'C:\\Program Files\\Intel\\iCLS Client\\',
 'C:\\Windows\\system32',
 'C:\\Windows',
 'C:\\Windows\\System32\\Wbem',
 'C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\',
 'C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common',
 'C:\\Program Files (x86)\\Intel\\Intel(R) Management Engine Components\\DAL',
 'C:\\Program Files\\Intel\\Intel(R) Management Engine Components\\DAL',
 'C:\\Program Files (x86)\\Intel\\Intel(R) Management Engine Components\\IPT',
 'C:\\Program Files\\Intel\\Intel(R) Management Engine Components\\IPT',
 'C:\\Program Files\\Intel\\WiFi\\bin\\',
 'C:\\Program Files\\Common Files\\Intel\\WirelessCommon\\',
 'C:\\WINDOWS\\system32',
 'C:\\WINDOWS',
 'C:\\WINDOWS\\System32\\Wbem',
 'C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0\\',
 'C:\\Program Files\\Java\\jdk1.8.0_112\\bin',
 'C:\\Program Files (x86)\\Skype\\Phone\\',
 '

In [7]:
import os
for k,v in os.environ.items():
    print(k,v)

SYSTEMROOT C:\WINDOWS
TMP C:\Users\John\AppData\Local\Temp
PROCESSOR_LEVEL 6
PLAT win32
HOMEDRIVE C:
PROCESSOR_IDENTIFIER Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
COMMONPROGRAMW6432 C:\Program Files\Common Files
PROGRAMW6432 C:\Program Files
GIT_PAGER cat
USERPROFILE C:\Users\John
MPLBACKEND module://ipykernel.pylab.backend_inline
PSMODULEPATH C:\Users\John\Documents\WindowsPowerShell\Modules;C:\Program Files\WindowsPowerShell\Modules;C:\WINDOWS\system32\WindowsPowerShell\v1.0\Modules\
IPY_INTERRUPT_EVENT 1124
COMPUTERNAME MSI
PROCESSOR_REVISION 5e03
USERDOMAIN_ROAMINGPROFILE MSI
JPY_INTERRUPT_EVENT 1124
TEMP C:\Users\John\AppData\Local\Temp
CLICOLOR 1
PROCESSOR_ARCHITEW6432 AMD64
SESSIONNAME Console
FPS_BROWSER_APP_PROFILE_STRING Internet Explorer
PROGRAMFILES C:\Program Files (x86)
CONFIGSETROOT C:\WINDOWS\ConfigSetRoot
PUBLIC C:\Users\Public
APR_ICONV_PATH C:\Program Files (x86)\Subversion\iconv
ONEDRIVE C:\Users\John\OneDrive
USERDOMAIN MSI
COMMONPROGRAMFILES C:\Program F

In [5]:
%%cython
#cython: boundscheck=False, wraparound=False, nonecheck=False

from libc.math cimport exp 
import numpy as np

def rbf_networkC(double[:, :] X,  double[:] beta, double theta):

    cdef int N = X.shape[0]
    cdef int D = X.shape[1]
    cdef double[:] Y = np.zeros(N)
    cdef int i, j, d
    cdef double r = 0

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

In [268]:
%%cython -a
#cython: embedsignature=True, initializedcheck=False
cimport cython

import math

import numpy as np
cimport numpy as np
from cpython cimport array
import array

TYPE = np.uint8
ctypedef np.uint8_t TYPE_t

cdef class Mat2:
    cpdef np.uint8_t[:,:] data
    def __init__(self, np.ndarray[TYPE_t, ndim=2] data):
        self.data = data
    
    cpdef get_data(self):
        return self.data
    
    def __str__(self):
        return "\n".join(str(list(r)) for r in self.data)
    
    cpdef Mat2 copy(self):
        return Mat2(np.asarray(self.data.copy()))
    
    cpdef Mat2 transpose(self):
        return Mat2(np.asarray(self.data).transpose().copy())
    
    cpdef int rows(self):
        return self.data.shape[0]
    
    cpdef int cols(self):
        return self.data.shape[1]
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cdef row_add(self, int r0, int r1):
        cdef int i
        cdef np.uint8_t[:] a, b
        a = self.data[r0]
        b = self.data[r1]
        for i in range(self.cols()):
            if a[i]:
                b[i] = 0 if b[i] else 1
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cdef col_add(self, int c0, int c1):
        cdef int i
        cdef np.uint8_t[:] d
        for i in range(self.rows()):
            d = self.data[i]
            if d[c0]:
                d[c1] = 0 if d[c1] else 1
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cpdef int gauss(self, int full_reduce=0, int blocksize=6):
        cdef int pivot_row, sec, i0, i1, r, p, r0, r1, rank, pcol, rows, cols
        cdef array.array pcols = array.array('i',[])
        cdef dict chunks
        cdef tuple t
        
        rows = self.rows()
        cols = self.cols()
        #pcols = array.array('i',[])
        pivot_row = 0
        for sec in range(math.ceil(float(cols) / blocksize)):
            i0 = sec * blocksize
            i1 = min(cols, (sec+1) * blocksize)
            # search for duplicate chunks of 'blocksize' bits and eliminate them
            chunks = dict()
            for r in range(pivot_row, rows):
                t = tuple(self.data[r][i0:i1])
                if not any(t): continue
                if t in chunks:
                    #print('hit (down)', r, chunks[t], t, i0, i1)
                    self.row_add(chunks[t], r)
                    #if x != None: x.row_add(chunks[t], r)
                    #if y != None: y.col_add(r, chunks[t])
                else:
                    chunks[t] = r
            p = i0
            while p < i1:
                for r0 in range(pivot_row, rows):
                    if self.data[r0][p] != 0:
                        if r0 != pivot_row:
                            self.row_add(r0, pivot_row)
                            #if x != None: x.row_add(r0, pivot_row)
                            #if y != None: y.col_add(pivot_row, r0)

                        for r1 in range(pivot_row+1, rows):
                            if pivot_row != r1 and self.data[r1][p] != 0:
                                self.row_add(pivot_row, r1)
                                #if x != None: x.row_add(pivot_row, r1)
                                #if y != None: y.col_add(r1, pivot_row)
                        if full_reduce: pcols.append(p)
                        pivot_row += 1
                        break
                p += 1
        rank = pivot_row
        if full_reduce:
            pivot_row -= 1
            for sec in range(math.ceil(self.cols() / blocksize) - 1, -1, -1):
                i0 = sec * blocksize
                i1 = min(self.cols(), (sec+1) * blocksize)
                # search for duplicate chunks of 'blocksize' bits and eliminate them
                chunks = dict()
                for r in range(pivot_row, -1, -1):
                    t = tuple(self.data[r][i0:i1])
                    if not any(t): continue
                    if t in chunks:
                        #print('hit (up)', r, chunks[t], t, i0, i1)
                        self.row_add(chunks[t], r)
                        #if x != None: x.row_add(chunks[t], r)
                        #if y != None: y.col_add(r, chunks[t])
                    else:
                        chunks[t] = r
                while len(pcols) and (i0 <= pcols[len(pcols)-1] < i1):
                    pcol = pcols.pop()
                    for r in range(0, pivot_row):
                        if self.data[r][pcol]:
                            self.row_add(pivot_row, r)
                            #if x != None: x.row_add(pivot_row, r)
                            #if y != None: y.col_add(r, pivot_row)
                    pivot_row -= 1
        return rank
    
    @cython.boundscheck(False)
    @cython.wraparound(False)
    cdef np.ndarray[TYPE_t,ndim=2] nullspace(self, int should_copy=1):
        """Returns a list of non-zero vectors that span the nullspace
        of the matrix. If the matrix has trivial kernel it returns the empty list."""
        cdef int cols, i, j
        cdef list nonpivots, pivots
        cdef Mat2 m
        cdef np.uint8_t[:] r
        cdef np.ndarray[TYPE_t,ndim=1] v
        cdef np.ndarray[TYPE_t,ndim=2] vectors
        if should_copy:
            m = self.copy()
        else:
            m = self
        m.gauss(full_reduce=True)
        cols = self.cols()
        nonpivots = list(range(cols))
        pivots = []
        for i, r in enumerate(m.data):
            for j in range(cols):
                if r[j]:
                    nonpivots.remove(j)
                    pivots.append(j)
                    break
        vectors = np.ndarray([0,cols], dtype=TYPE)
        for i in nonpivots:
            v = np.zeros(cols,dtype=TYPE)
            v[i] = 1
            for r, j in zip(m.data, pivots):
                if r[i]: v[j] = 1
            vectors = np.insert(vectors, vectors.shape[0], v, 0)
        return vectors

cpdef ident(n):
    i = np.identity(n, TYPE)
    return Mat2(i)
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef Mat2 xi(Mat2 m, np.ndarray[TYPE_t,ndim=1] z):
    cdef int rows, alpha, beta, gamma, v1,v2, i, j
    cdef np.ndarray[TYPE_t, ndim=1] ra, rb, rab, rg, rag, rbg
    cdef np.ndarray[TYPE_t, ndim=2] data
    cdef np.uint8_t[:] r
    rows = m.rows()
    data = np.ndarray([0,m.cols()], dtype=TYPE)
    for alpha in range(rows):
        ra = np.asarray(m.data[alpha])
        for beta in range(alpha+1, rows):
            rb = np.asarray(m.data[beta])
            rab = ra*rb
            #rab = [i*j for i,j in zip(ra,rb)]
            for gamma in range(beta+1, rows):
                rg = np.asarray(m.data[gamma])
                rag = ra*rg
                rbg = rb*rg
                #rag = [i*j for i,j in zip(ra,rg)]
                #rbg = [i*j for i,j in zip(rb,rg)]
                
                if z[alpha]:
                    if not z[beta]:
                        if not z[gamma]:
                            data = np.insert(data, data.shape[0], rbg, 0)
                            #data.append(rbg)
                            continue
                        data = np.insert(data, data.shape[0], (rbg+rab)%2, 0)
                        #data.append([0 if v1==v2 else 1 for v1,v2 in zip(rbg,rab)])
                        continue
                    elif not z[gamma]:
                        data = np.insert(data, data.shape[0], (rbg+rag)%2, 0)
                        #data.append([0 if v1==v2 else 1 for v1,v2 in zip(rbg,rag)])
                        continue
                    else: #z[alpha], z[beta] and z[gamma] are all true
                        data = np.insert(data, data.shape[0], (rbg+rab+rag)%2, 0)
                        #r = [0 if v1==v2 else 1 for v1,v2 in zip(rab,rag)]
                        #data.append([0 if v1==v2 else 1 for v1,v2 in zip(r,rbg)])
                        continue
                elif z[beta]:
                    if z[gamma]:
                        data = np.insert(data, data.shape[0], (rab+rag)%2, 0)
                        #data.append([0 if v1==v2 else 1 for v1,v2 in zip(rab,rag)])
                        continue
                    data = np.insert(data, data.shape[0], rag, 0)
                    #data.append(rag)
                    continue
                elif z[gamma]:
                    data = np.insert(data, data.shape[0], rab, 0)
                    #data.append(rab.copy())
                    continue
    for r in m.data: 
        data = np.insert(data, data.shape[0], np.asarray(r).copy(), 0)
        #data.append(r.copy())            
    return Mat2(data)

cdef class ToddReturn:
    cpdef int a, b
    cpdef np.uint8_t[:] z, y
    def __init__(self, int a, int b, np.ndarray[TYPE_t, ndim=1] z, np.ndarray[TYPE_t, ndim=1] y):
        self.a = a
        self.b = b
        self.z = z
        self.y = y

@cython.boundscheck(False)
@cython.wraparound(False)
cdef ToddReturn find_todd_match(Mat2 m):
    cdef int rows, cols, a, b, i
    cdef np.ndarray[TYPE_t,ndim=1] z, y, r
    cdef np.ndarray[TYPE_t,ndim=2] options
    cdef Mat2 bigm
    rows = m.rows()
    cols = m.cols()
    for a in range(cols):
        for b in range(a+1, cols):
            z = np.zeros(rows, dtype=TYPE)
            #z = [0]*rows
            for i in range(rows):
                r = np.asarray(m.data[i])
                if r[a]:
                    if not r[b]:
                        z[i] = 1
                else:
                    if r[b]:
                        z[i] = 1
            bigm = xi(m, z)
            #print(bigm, '.')
            options = bigm.nullspace(should_copy=0)
            #print(bigm)
            for y in options:
                if y[a] + y[b] == 1: return ToddReturn(a,b,z,y)

    return ToddReturn(-1,-1,z,y)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef int remove_trivial_cols(Mat2 m):
    cdef int newcols, should_break, a, b
    while True:
        newcols = m.rows()
        for a in range(newcols):
            if not any(m.data[a]):
                m.data = np.delete(m.data, a, 0)
                #m.data.pop(a)
                break
            should_break = 0
            for b in range(a+1, newcols):
                if np.alltrue(np.asarray(m.data[a]) == np.asarray(m.data[b])):
                    m.data = np.delete(m.data, b, 0)
                    m.data = np.delete(m.data, a, 0)
                    #m.data.pop(b)
                    #m.data.pop(a)
                    should_break = 1
                    break
            if should_break: break
        else: # Didn't break out of for-loop so didn't find any match
            break
    return newcols

@cython.boundscheck(False)
@cython.wraparound(False)
cdef Mat2 do_todd_single(Mat2 m):
    cdef int a, b, i
    cdef np.ndarray[TYPE_t, ndim=1] z, y, 
    cdef np.uint8_t[:] c
    cdef ToddReturn treturn
    #startcols = m.cols()
    treturn = find_todd_match(m)
    a = treturn.a
    b = treturn.b
    z = np.asarray(treturn.z)
    y = np.asarray(treturn.y) 
    if a == -1: return m
    m = m.transpose()
    #odd_y = sum(y) % 2
    for i,c in enumerate(m.data):
        if not y[i]: continue
        for j in range(len(c)):
            if z[j]: c[j] = 0 if c[j] else 1
    if sum(y) % 2 == 1:
        m.data = np.insert(m.data, m.data.shape[0], z, 0)
        #m.data.append(z)
    m.data = np.delete(m.data,b,0)
    m.data = np.delete(m.data,a,0)
    #m.data.pop(b)
    #m.data.pop(a)
    
    remove_trivial_cols(m)
                
    return m.transpose()

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef Mat2 todd_iter(Mat2 m, int quiet=1):
    cdef int oldcols, newcols
    m = m.transpose()
    remove_trivial_cols(m)
    m = m.transpose()
    oldcols = m.cols()
    while True:
        m = do_todd_single(m)
        newcols = m.cols()
        if oldcols == newcols:
            return m
        if not quiet: print(oldcols-newcols, end='.')
        oldcols = newcols
        

cpdef f():
    cdef Mat2 a
    cdef np.ndarray[TYPE_t, ndim=2] l
    l = np.random.randint(2,size=[50,200],dtype="uint8")
    a = Mat2(l)
    #print(a.data[0][0])
    a.gauss(full_reduce=True)
    


In [214]:
s = """
[ 1 1 1 1 0 0 0 1 1 0 1 ]
[ 1 1 0 0 1 0 0 1 0 1 1 ]
[ 1 0 1 0 0 1 1 1 0 1 0 ]
[ 1 0 0 1 1 0 1 0 0 0 1 ]"""
l = s.splitlines()[1:]
l = [[int(i) for i in j[1:-1].split(" ") if i] for j in l]

In [270]:
l = np.random.randint(2,size=[7,40],dtype="uint8")
c = np.asarray(l,dtype=TYPE)
a = Mat2(c)
#print(str(a),'\n\n')
#b = todd_iter(a)
# print(str(b))
def f():
    l = np.random.randint(2,size=[9,60],dtype="uint8")
    c = np.asarray(l,dtype=TYPE)
    a = Mat2(c)
    todd_iter(a)
%timeit f()

1.31 s ± 262 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [220]:
print(str(a.transpose()))

[1, 1, 1, 1]
[1, 1, 0, 0]
[1, 0, 1, 0]
[1, 0, 0, 1]
[0, 1, 0, 1]
[0, 0, 1, 0]
[0, 0, 1, 1]
[1, 1, 1, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]
[1, 1, 0, 1]


In [200]:
list(enumerate(a.get_data()))

[(0, <MemoryView of 'ndarray' at 0x7f77e614ea38>),
 (1, <MemoryView of 'ndarray' at 0x7f77e614e398>)]

In [207]:
d = np.ndarray(0,dtype="uint8")
d.resize([5,3])
d2 = d.transpose()
d2[0,0] = 1
print(d)

[[1 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [20]:
%%cython -a

cdef class GraphS:
    cdef int vindex, nedges
    cdef dict graph, ty
    def __init__(self):
        self.graph = dict()
        self.ty = dict()
        self.vindex = 0
        self.nedges = 0

    cdef add_vertices(self, int amount):
        cdef int i
        for i in range(self.vindex, self.vindex + amount):
            self.graph[i] = dict()
            self.ty[i] = 0
        self.vindex += amount

    cdef add_vertex(self):
        self.add_vertices(1)

    cdef add_edges(self, list edges):
        cdef int s,t
        for s,t in edges:
            self.nedges += 1
            self.graph[s][t] = True
            self.graph[t][s] = True

    cdef add_edge(self, tuple edge):
        self.add_edges([edge])

    cdef remove_vertices(self, list vertices):
        cdef int v, v1
        cdef list vs
        for v in vertices:
            vs = list(self.graph[v])
            # remove all edges
            for v1 in vs:
                self.nedges -= 1
                del self.graph[v][v1]
                del self.graph[v1][v]
            # remove the vertex
            del self.graph[v]
            del self.ty[v]

    cdef remove_vertex(self, int vertex):
        self.remove_vertices([vertex])

    cdef remove_solo_vertices(self):
        '''Deletes all vertices that are not connected to any other vertex.
        Should be replaced by a faster alternative if available in the backend'''
        self.remove_vertices([v for v in self.vertices() if self.get_vertex_degree(v)==0])

    cdef remove_edges(self, list edges):
        cdef int s,t
        for s,t in edges:
            self.nedges -= 1
            del self.graph[s][t]
            del self.graph[t][s]

    #cdef remove_edge(self, tuple edge):
    #    self.remove_edge([edge])

    cdef int num_vertices(self):
        return len(self.graph)

    cdef int num_edges(self):
        return self.nedges

    cdef vertices(self):
        return self.graph.keys()

    cdef list edges(self):
        cdef int v0, v1
        cdef dict adj
        cdef list output = []
        for v0,adj in self.graph.items():
            for v1 in adj:
                if v1 > v0: output.append((v0,v1))
        return output

    cdef set edge_set(self):
        '''Returns a set of indices of edges. Should be overloaded if the backend
        supplies a cheaper version than this.'''
        return set(self.edges())

    cdef inline tuple edge_st(self, tuple edge):
        return edge

    cdef get_neighbours(self, int vertex):
        return self.graph[vertex].keys()

    cdef int get_vertex_degree(self, int vertex):
        return len(self.graph[vertex])

    cdef list get_incident_edges(self, int vertex):
        return [(vertex, v1) if v1 > vertex else (v1, vertex) for v1 in self.graph[vertex]]

    cdef bint is_connected(self,int v1,int v2):
        return v2 in self.graph[v1]

    cdef int get_type(self, int vertex):
        return self.ty[vertex]

    cdef dict get_types(self):
        return self.ty

    cdef set_type(self, int vertex, int t):
        self.ty[vertex] = t

cdef list match_bialg_parallel(GraphS g, int num=-1):
    cdef set candidates = g.edge_set()
    cdef dict types = g.get_types()
    cdef int i = 0
    cdef int v0,v1,v0t,v1t,n
    cdef tuple c
    cdef list m = []
    while (num == -1 or i < num) and len(candidates) > 0:
        v0, v1 = g.edge_st(candidates.pop())
        v0t = types[v0]
        v1t = types[v1]
        if ((v0t == 1 and v1t == 2) or (v0t == 2 and v1t == 1)):
            v0n = [n for n in g.get_neighbours(v0) if not n == v1]
            v1n = [n for n in g.get_neighbours(v1) if not n == v0]
            if (
                all([types[n] == v1t for n in v0n]) and
                all([types[n] == v0t for n in v1n])):
                i += 1
                for v in v0n:
                    for c in g.get_incident_edges(v): candidates.discard(c)
                for v in v1n:
                    for c in g.get_incident_edges(v): candidates.discard(c)
                m.append([v0,v1,v0n,v1n])
    return m


cdef bialg(GraphS g, list matches):
    cdef list m, es, del_verts, add_edges, del_edges
    cdef tuple e
    del_verts = []
    add_edges = []
    del_edges = []
    for m in matches:
        del_verts.append(m[0])
        del_verts.append(m[1])
        es = [(i,j) for i in m[2] for j in m[3]]
        for e in es:
            if g.is_connected(e[0], e[1]): del_edges.append(e)
            else: add_edges.append(e)
    
    g.remove_edges(del_edges)
    g.add_edges(add_edges)
    g.remove_vertices(del_verts)
    g.remove_solo_vertices()

cpdef GraphS zigzag(int sz):
    cdef int i
    g = GraphS()
    g.add_vertices(2*sz+4)
    for i in range(1,sz+1):
        g.set_type(2*i, (i%2)+1)
        g.set_type(2*i+1, (i%2)+1)
    g.add_edges([(0,2),(1,3)])
    g.add_edges([(2*i,2*i+2) for i in range(1,sz)])
    g.add_edges([(2*i,2*i+3) for i in range(1,sz)])
    g.add_edges([(2*i+1,2*i+2) for i in range(1,sz)])
    g.add_edges([(2*i+1,2*i+3) for i in range(1,sz)])
    g.add_edges([(2*sz,2*sz+2),(2*sz+1,2*sz+3)])
    return g

cpdef total_reduce(GraphS g):
    print("normalising ZX diagram " + str(g))
    cdef int it
    cdef list m
    it = 0
    while True:
        it += 1
        m = match_bialg_parallel(g)
        if len(m) != 0: bialg(g, m)
        else: break
    print("completed in " + str(it) + " iterations")

In [58]:
g = zigzag(1000000)
%time total_reduce(g)

normalising ZX diagram <_cython_magic_7f300cec609a215475ad974fdc518235.GraphS object at 0x7f8b37d1f708>
completed in 18 iterations
CPU times: user 17.2 s, sys: 1.42 s, total: 18.6 s
Wall time: 18.6 s


In [4]:
from scipy.interpolate import Rbf
rbf = Rbf(X[:,0], X[:,1], X[:,2], X[:,3], X[:, 4], np.random.rand(N))
Xtuple = tuple([X[:, i] for i in range(D)])

NameError: name 'X' is not defined

In [16]:
%timeit rbf(Xtuple)

435 ms ± 8.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
